## Welcome to the PSIG Short Course on Centrifugal Compressors and Stations

- This notebook accompanies the <a href="https://psig.org/conference/pipeline-simulation-short-course/">Pipeline Simulation Short Course</a>

- The source code is available at <a href="https://github.com/Cody-at-SolarTurbines/PSIG-Compressor-Course">Cody-at-SolarTurbines</a>

In [ ]:
# special command for plotting
%matplotlib widget

# standard python modules
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from scipy.linalg import solve, inv
import json

## Exercise 5

---

<font size="4">
Suppose the following data set is obtained from a test cell:
    
<br/>    

q = [3945, 2692, 4751, 6263, 5802] in ACFM 

p1 = [800, 801, 809, 806, 805] in psia
    
p2 = [676, 681, 680, 669, 671] in psia
    
T1 = [81, 81, 79, 80, 78, 80 ] in degF
    
Z1 = [0.966, 0.964, 0.963, 0.964, 0.965 ] unitless
    
SG = [0.555, 0.554, 0.556, 0.552, 0.555] unitless

Estimate 𝛼 using a <a href="https://mathworld.wolfram.com/LeastSquaresFitting.html">Least Squares</a> approach for equation: $Q = \alpha \sqrt{\frac{(p_1 - p_2) T_1 Z_1}{ p_1 SG}} $
</font>

---

## Solution 5:

<br/>

There are many ways to estimate $\alpha$.  We can do this numerically or calculate directly.  We do both below.

<br/><br/>

###  We will use Numpy and Scipy to implement the solution

These are python packages for scientific computing
- https://numpy.org/
- https://scipy.org/

### Import data

- Import the data and convert the data to Numpy arrays for computation

In [ ]:
# Import data from a json file
with open('data.json', 'r') as openfile:
    data = json.load(openfile)
    
# convert data lists to numpy arrays
for k,v in data.items():
    data[k] = np.array(v)
    
    # t1 variable is in degF and we need degR
    if k == 't1':
        data[k] += 460

In [ ]:
# create a DP variable
data['p_diff'] = data['p1']-data['p2']

# setup data in array
data_raw = np.vstack((data['p1'],data['p2'],data['t1'],
                      data['z1'],data['sg'],data['p_diff'],data['q']))

# sort by differential pressure (this will make for nice connected points on the plot)
data_sorted = data_raw[:,data_raw[5, :].argsort()]

# setup x_data and y_data
x_data = data_sorted[:5,:]
y_data = data_sorted[6,:]

#### What just happened above?

1) create a numpy array of the individual arrays

In [ ]:
data_raw

2) sort the entire array by the sort of the 6th row (corresponds to p_diff variable)

In [ ]:
print('Here are the values:\t\t\t',data_raw[5, :])
print('Here is the index sort values\t\t',data_raw[5, :].argsort())

3)  Here is the sorted data array

In [ ]:
data_sorted

4) Down select for x_data only

In [ ]:
x_data

## Visualize the data

In [ ]:
f, ax = plt.subplots(figsize=(9,7))
ax.plot(data_sorted[5,:], data_sorted[6,:], 'o', markersize=10, label='Raw Data Points')
ax.set_xlabel('DP [psia]')
ax.set_ylabel('Q [ACFM]')
ax.set_title('Raw Data for Exercise 5')
ax.grid()
ax.legend()

## Now create the flow function from the equation above

In [ ]:
# create a function y = f(x_data,alpha)
# Note that x_data[0] = p1; x_data[1] = p2; and so on as above

def flow_q(x_data,alpha):
    """ Flow function from equation """
    return alpha*np.sqrt((x_data[0]-x_data[1])*x_data[2]*x_data[3]/(x_data[0]*x_data[4]))

### Use Scipy `curve_fit` method to find the optimal alpha value

- initial guess of 1000

In [ ]:
help(curve_fit)

In [ ]:
# popt = optimal coefficient
# pcov = covariance of popt

popt, pcov = curve_fit(flow_q,x_data,y_data,1000)
alpha = popt[0]
print(f'>>> The optimal alpha value is: {alpha}')

### Now predict the flow, based on the optimal $\alpha$ and the given inputs

In [ ]:
# we will call the estimated or predicted flow, yhat

yhat = flow_q(x_data,alpha) 

## Visualize the data and the fit

In [ ]:
f, ax1 = plt.subplots(figsize=(9,7))
ax1.plot(data_sorted[5,:], data_sorted[6,:], 'o', markersize=10, label='Raw Data Points')
ax1.plot(data_sorted[5,:], yhat, '-s', label='Optimal Fit')
ax1.set_xlabel('DP [psia]')
ax1.set_ylabel('Q [ACFM]')
ax1.set_title('Raw Data for Exercise 5')
ax1.set_title(r'$\alpha = $' + f'{alpha:.3f}')
ax1.grid()
ax1.legend()

### If we need $\beta$, we can solve for it since $\alpha = \sqrt{1/\beta} \Leftrightarrow \beta = 1/\alpha^2$

In [ ]:
beta = 1/alpha**2
print(f'Beta = {beta:.10f}')

## Solution 5 (Bonus)

$\alpha$ can be calculated directly with <a href="https://mathworld.wolfram.com/LeastSquaresFitting.html">sums</a> or <a href="https://www.khanacademy.org/math/linear-algebra/alternate-bases/orthogonal-projections/v/linear-algebra-least-squares-examples">linear algebra</a>, instead of using a numerical solver like the Levenberg-Marquardt, which is the default in `curve_fit`.

We will use linear algebra.  We want to put the data in the form $Ax = b$ and solve for $x$.  Note that the equation is of the form $Q_i=\alpha K_i$, where $i$ is the $i$th observation and $K_i$ is constant in the input parameters!

Therefore, we can simply solve the <a href="https://mathworld.wolfram.com/NormalEquation.html">Normal Equation</a>: $x^* = (A^TA)^{-1}A^Tb = \alpha$

All that we need to do is create the matrix $A$ (which is 1-dimensional in this case)

In [ ]:
# setup matrix A

A = np.array([flow_q(x_observation,1) for x_observation in x_data.T]).reshape((5,1))

In [ ]:
# solve the normal equation

alpha_normal = np.dot(np.dot(inv(np.dot(A.T,A)),A.T),y_data)

print(f'Normal Equation alpha:\t\t\t {alpha_normal[0]}')
print(f'Numerically solved for alpha:\t\t {alpha}')